In [20]:
import pandas as pd
from os import getcwd

In [23]:
path = getcwd()
pd.read_csv(path+'/../output/results_df.csv')

,game_date,team,opponent,record,1st_inning,2nd_inning,3rd_inning,4th_inning,5th_inning,6th_inning,7th_inning,8th_inning,9th_inning,runs,hits,errors,result
0,2023-02-25,Mets,Astros,test,2,0,0,0,0,0,0,0,0,2,4,4,test
1,2023-02-25,Astros,Mets,test,0,0,0,0,0,1,3,0,X,4,8,0,test
2,2023-02-25,Nationals,Cardinals,test,1,0,0,0,0,0,0,1,1,3,9,1,test
3,2023-02-25,Cardinals,Nationals,test,0,0,2,0,0,0,0,0,0,2,5,0,test
4,2023-02-25,Blue Jays,Pirates,test,0,0,1,3,2,0,3,0,0,9,13,1,test
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,2023-02-26,Angels,White Sox,test,1,3,3,0,0,0,0,0,X,7,13,0,test
62,2023-02-26,Brewers,Rockies,test,1,2,0,0,0,1,0,2,0,6,10,2,test
63,2023-02-26,Rockies,Brewers,test,0,2,1,0,0,0,5,4,X,12,12,0,test
64,2023-02-26,Athletics,Brewers,test,4,0,1,0,0,1,5,0,0,11,13,0,test
